Toje pačioje direktorijoje turi būti katalogi "initial" (kuriame 'train_data.txt' ir 'test_data_solution.txt' ) ir tuščio aplankalo 'cleaned_data' (failų įrašynėjimui)

In [1]:
# ISANKSTINIS DUOMENU APDOROJIMAS (Data processing)

# Nuskaitome train_data.txt faila importuojame biblioteka.

### engine='python', siuo atveju nes bandytu su c kalba skaityt del efektyvumo ir warninga mestu
### sep='' parodo kokiais simboliais stulpeliai atskirti
### columns_name stulpeliu pavadinimai kuriuos uzvadinsim, jie aprasyti descriptions.txt faile
### names parametras leidzia nustatyt stulpeliu pavadinimus ant skaitymo


import pandas as pd

column_names = ["ID", "TITLE", "GENRE", "DESCRIPTION"]
movies_df = pd.read_csv('initial/train_data.txt', sep=':::', engine='python', names=column_names)


In [2]:
movies_df

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on ...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The sist...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and hav...


In [3]:
#Patikriname, ar nėra trūkstamų reikšmių:

print("\nTrūkstamų reikšmių skaičius Movies DataFrame:")
print(movies_df.isnull().sum())


Trūkstamų reikšmių skaičius Movies DataFrame:
ID             0
TITLE          0
GENRE          0
DESCRIPTION    0
dtype: int64


In [3]:
movies_df = movies_df.drop_duplicates(subset='DESCRIPTION') # Dublikatu pasalinimas.

In [4]:
# Pasiziurime kiek eiluciu yra pasalinus duplikatus:
len(movies_df)
# 54 086 eiluciu ant traininimo data'os

54086

In [5]:
# NORMALIZAVIMAS

# Funkcijos kurias reikes ir test data'ai pritaikyti

import string
import re
from nltk.corpus import stopwords

stopwords_initial = stopwords.words('english')
stopwords_cleaned = [word.translate(str.maketrans('', '', string.punctuation)) for word in stopwords_initial] # Pasalinami simboliai is stopwords nes is teksto anksciau pasalinam, pvz '

# Funkcija 'Nereiksmingu zodziu pasalinimas':
def remove_stopwords(words):
    filtered_words = [word for word in words if word not in stopwords_cleaned]
    return filtered_words

# Funkcija 'Skyrybos zenklu pasalinimas':
def remove_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Funkcija 'HTML tagu pasalinimas':
def remove_html_tags(text):
    if isinstance(text, str):
        return re.sub('<.*?>', ' ', text)
    return text

# Funkcija 'Specialiu simboliu ir pridetiniu laukeliu (spaces)pasalinimas':
def remove_multiple_whitespaces(text):
    return re.sub('\\s+', ' ', text)                                          

# Funkcija apjungianti auksciau aprasytas funkcijas bei tokenizacija ir splitinima:
def clean_df(df, column):
    df_clean = df.copy()
    df_clean.loc[:, column] = df_clean[column].apply(remove_html_tags)  # Naudojamas .loc kad isvengti warningu del nenumatyto pandas poveikio
    df_clean.loc[:, column] = df_clean[column].apply(lambda x: x.lower()) # Visas raides pavercia i mazasias raides
    df_clean.loc[:, column] = df_clean[column].apply(remove_punctuations)
    df_clean.loc[:, column] = df_clean[column].apply(remove_multiple_whitespaces)
    df_clean.loc[:, column] = df_clean[column].str.split() # Tokenizacija, teksto skaidymas i vienetus 
    df_clean.loc[:, column] = df_clean[column].apply(remove_stopwords)
    return df_clean


In [6]:
df = clean_df(movies_df, "DESCRIPTION") # Cia atlieka visa pilna sutvarkyma, galima bus perpanaudot testavimo aprasymams

In [7]:
# Sutvarkytas duomenu failas.
df

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,"[listening, conversation, doctor, parents, 10y..."
1,2,Cupid (1997),thriller,"[brother, sister, past, incestuous, relationsh..."
2,3,"Young, Wild and Wonderful (1980)",adult,"[bus, empties, students, field, trip, museum, ..."
3,4,The Secret Sin (1915),drama,"[help, unemployed, father, make, ends, meet, e..."
4,5,The Unrecovered (2007),drama,"[films, title, refers, unrecovered, bodies, gr..."
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,"[shortlived, nbc, live, sitcom, centered, boni..."
54210,54211,Dead Girls Don't Cry (????),horror,"[next, generation, exploitation, sisters, kapa..."
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"[ze, bestaan, echt, standup, comedy, growing, ..."
54212,54213,Make Your Own Bed (1944),comedy,"[walter, vivian, live, country, difficult, tim..."


In [8]:
X_stemming = df['DESCRIPTION'] # Aibe kuria modelis priims kaip X (Filmu aprasymai)
y = df['GENRE'] # Aibe kuria bandys gauti (zanrai)

X_lemmatization = X_stemming.copy() # Kopija lemavimui 

In [9]:
# Stemingas (stemming), pasalina zodziu galunes ar pradines dalis:
import nltk
from nltk.tokenize import word_tokenize

def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= [ps.stem(word) for word in text]
    return text

X_stemming = X_stemming.apply(simple_stemmer)

In [10]:
# Lematizavimas (lemmatization), žodis sumažinamas iki jo lingvistinės pagrindinės formos):

from nltk.stem import WordNetLemmatizer

def simple_lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    return text

X_lemmatization = X_lemmatization.apply(simple_lemmatizer)

In [11]:
# Sujungiam stemingo ir lematizacijos rezultatus su etiketėmis į vieną DataFrame:

stemming = pd.concat([X_stemming, y], axis=1)
lemmatization = pd.concat([X_lemmatization, y], axis=1)

# Parametras axis=1 nurodo, kad sujungimas vyksta išilgai stulpelių, o ne eilučių.

In [12]:
stemming

,DESCRIPTION,GENRE
0,"[listen, convers, doctor, parent, 10yearold, o...",drama
1,"[brother, sister, past, incestu, relationship,...",thriller
2,"[bu, empti, student, field, trip, museum, natu...",adult
3,"[help, unemploy, father, make, end, meet, edit...",drama
4,"[film, titl, refer, unrecov, bodi, ground, zer...",drama
...,...,...
54209,"[shortliv, nbc, live, sitcom, center, bonino, ...",comedy
54210,"[next, gener, exploit, sister, kapa, bay, soro...",horror
54211,"[ze, bestaan, echt, standup, comedi, grow, fac...",documentary
54212,"[walter, vivian, live, countri, difficult, tim...",comedy


In [13]:
lemmatization

,DESCRIPTION,GENRE
0,"[listening, conversation, doctor, parent, 10ye...",drama
1,"[brother, sister, past, incestuous, relationsh...",thriller
2,"[bus, empty, student, field, trip, museum, nat...",adult
3,"[help, unemployed, father, make, end, meet, ed...",drama
4,"[film, title, refers, unrecovered, body, groun...",drama
...,...,...
54209,"[shortlived, nbc, live, sitcom, centered, boni...",comedy
54210,"[next, generation, exploitation, sister, kapa,...",horror
54211,"[ze, bestaan, echt, standup, comedy, growing, ...",documentary
54212,"[walter, vivian, live, country, difficult, tim...",comedy


In [14]:
# Isirasome sutvarkytus failus i anksciau sukurta tuscia kataliga, kad galėtume panaudoti modeliuose
stemming.to_csv("cleaned_data/stemming.csv", index=False)
lemmatization.to_csv("cleaned_data/lemmatization.csv", index=False)

Nuo čia bus kiti failai naudojami, kad daug kur perpanaudot jau sutvarkytus stemingus ir lematizavimus.